Top 1000 Movies by IMDB Rating [data](https://www.kaggle.com/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows)

- PosterLink: Link of the poster that imdb using
- SeriesTitle: Name of the movie 
- ReleasedYear: Year at which that movie released
- Certificate: Certificate earned by that movie
- Runtime: Total runtime of the movie
- Genre: Genre of the movie
- IMDB Rating: Rating of the movie at IMDB site
- Overview: mini story/ summary
- Meta_score: Score earned by the movie
- Director: Name of the Director
- Star1,Star2,Star3,Star4: Name of the Stars
- No of votes: Total number of votes
- Gross: Money earned by that movie

In [ ]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))
import numpy as np # linear algebra
print("NumPy version: {}". format(np.__version__))
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
print("pandas version: {}". format(pd.__version__))
import matplotlib # collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings # ignore warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
path = '../input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv'
df = pd.read_csv(path, delimiter=',')
df.shape

In [ ]:
df.head(3)

In [ ]:
df.columns

In [ ]:
df.drop('Star1', axis =1, inplace = True)
df.drop('Star2', axis =1, inplace = True)
df.drop('Star3', axis =1, inplace = True)
df.drop('Star4', axis =1, inplace = True)

In [ ]:
df.describe()

In [ ]:
df.isna().sum() # missing values

In [ ]:
mode = df["Certificate"].mode()
mode

In [ ]:
df['Certificate'].fillna(df['Certificate'].mode()[0], inplace=True)
df.isna().sum() # missing values

In [ ]:
df['Meta_score'].fillna(10*df['IMDB_Rating'], inplace=True)
df.isna().sum() # missing values

In [ ]:
df.dropna(subset=['Gross'], how='all', inplace=True)
df.shape

In [ ]:
df.isna().sum() # missing values

In [ ]:
# outliers using boxplot
df.describe()

In [ ]:
df['Released_Year'].unique()

In [ ]:
df = df.drop(df[df['Released_Year'] == 'PG'].index)
df['Released_Year'].unique()

In [ ]:
df['IMDB_Rating'].unique()

In [ ]:
df['Genre'].unique()

In [ ]:
df.dtypes

In [ ]:
df.drop('Poster_Link',axis=1,inplace=True)
df.drop('Series_Title',axis=1,inplace=True)
df.drop('Overview',axis=1,inplace=True)

In [ ]:
df['Released_Year'] = df['Released_Year'].astype(int)
df = df.astype({"Runtime": str})
df['Runtime']=df['Runtime'].str.replace(' min','') # Runtime = 142 min
df = df.astype({"Runtime": int})
df = df.astype({"Gross": str})
df['Gross']=df['Gross'].str.replace(',','')
df["Gross"] = df["Gross"].apply(pd.to_numeric) # convert gross to numeric
# df = df.astype({"Gross": int})
# df = df.apply(pd.to_numeric, errors='ignore')
df.dtypes

In [ ]:
# distribution
ax = sns.distplot(df['Gross']) # histogram distribution

In [ ]:
# distribution
ax = sns.distplot(df['Meta_score']) # histogram distribution

In [ ]:
# distribution
ax = sns.distplot(df['IMDB_Rating']) # histogram distribution

In [ ]:
obj=['Certificate','Genre','Director']

In [ ]:
cols_to_std = ['Runtime','Released_Year','IMDB_Rating','Meta_score','No_of_Votes'] # Meta_score or Gross?

In [ ]:
target = df['Gross'] # Meta_score or Gross?

In [ ]:
# Creating dummy variables
dummies = pd.get_dummies(df[obj],drop_first=True)
df_dummies = pd.concat([df[cols_to_std],dummies],axis=1)
df_dummies.dtypes

In [ ]:
df_dummies.head(3)

In [ ]:
df.columns

In [ ]:
df.drop('Certificate',axis=1,inplace=True)
df.drop('Runtime',axis=1,inplace=True)
df.drop('Genre',axis=1,inplace=True)
df.drop('Director',axis=1,inplace=True)
df.drop('No_of_Votes',axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
top = 15
corr = df.corr()
top15 = corr.nlargest(top, 'Gross')['Gross'].index
corr_top15 = df[top15].corr()
f,ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr_top15, square=True, ax=ax, annot=True, cmap='coolwarm', fmt='.2f', annot_kws={'size':12})
plt.title('Top correlated features of dataset', size=16)
plt.show()

In [ ]:
# Splitting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,target,test_size=0.20) # df_dummies

In [ ]:
# Standardizing the data
cols_to_std = ['Released_Year','IMDB_Rating','Meta_score'] # Meta_score or Gross?
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train[cols_to_std])
X_train[cols_to_std] = scaler.transform(X_train[cols_to_std])
X_test[cols_to_std] = scaler.transform(X_test[cols_to_std])

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
model = XGBRegressor()
model.fit(X_train, y_train)
Y_pred = model.predict(X_test)
score = model.score(X_train, y_train)
print('Training Score:', score)
score = model.score(X_test, y_test)
print('Testing Score:', score)
output = pd.DataFrame({'Predicted':Y_pred})

In [ ]:
output.head(3)

score is 99%